# PoPS Border simulation command line examples

## A trivial example

First, we create a [YAML](https://www.codeproject.com/Articles/1214409/Learn-YAML-in-five-minutes) configuration file
with minimal (and unrealistic) parameters to run the simulation (we use `simple.yml` as the file name and we place it under the `data` directory, so `data/simple.yml`; the `/` as directory separator works on all operating systems):

In [1]:
%%writefile data/simple.yml
# In the configuration of the shipment generator, we only use couple of ports, flowers, and ports
# because they are currently not used by the simulation anyway.
consignment:
  generation_method: parameter_based
  parameter_based:
    origins:
    - Netherlands
    - Mexico
    flowers:
    - Hyacinthus
    - Rosa
    ports:
    - NY JFK CBP
    - FL Miami Air CBP
    # We set very low number of boxes and items.
    boxes:
      min: 2
      max: 5
  items_per_box:
    default: 10
contamination:
  # We will contaminate individual items.
  contamination_unit: item
  # We use one of the simplest contaminant placement options.
  # Fixed contamination rate for each consignment 
  contamination_rate:
    distribution: fixed_value
    value: 0.05
  # Contaminant present in a portion of random boxes
  arrangement: random_box
  random_box:
    # Contaminants present in every second consignment on average
    probability: 0.5
    # Roughly a third of boxes should be contaminated
    ratio: 0.33
inspection:
  # Decision about sampling will be made using items as a inspection unit.
  unit: items
  # We will inspect 2% of items.
  sample_strategy: proportion
  # We allow no boxes to be inspected.
  min_boxes: 0
  # No contamination tolerance threshold for this example, set to 0
  tolerance_level: 0
  proportion:
    value: 0.02
  # Once we pick a box, we will inspect all of its content.
  within_box_proportion: 1
  # Boxes for inspection are selected randomly.
  selection_strategy: random

Writing data/simple.yml


Then we create run the simulation generating only couple consignments.
The low number of boxes per consignment (`boxes`) and items per box (`items_per_box`) in the configuration and the low number
of consignments in the simulation allows us to show the individual items
within each consignment using a symbol of a flower (`✿`) indicating ones with contaminants using a symbol of a bug (`🐛`) instead of a flower.
Change the seed value to see different random placements of the contaminants within the consignment
(for example, using `--seed=5`).
The `arrangement` is set to `random_box`, so you will always see contaminant present on the whole box
which is marked by pipe symbols (`|`) in the output.
We are using `python -m popsborder` to run the simulation through its command line interface:

In [2]:
!python -m popsborder --num-consignments=5 --config-file=data/simple.yml --pretty --seed=4

━━ Consignment ━━ Boxes: 5 ━━ Items: 50 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Consignment ━━ Boxes: 3 ━━ Items: 30 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Consignment ━━ Boxes: 4 ━━ Items: 40 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Consignment ━━ Boxes: 3 ━━ Items: 30 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Consignment ━━ Boxes: 4 ━━ Items: 40 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿


Simulation parameters:
----------------------------------------------------------
consignments:
	 Number consignments simulated: 5
	 Avg. number of boxes per consignment: 4
	 Avg. numbe

## A more complex example

We start again by preparing the configuration file. In this example, we use more realistic size of the consignments
by setting `boxes` and `items_per_box`. We also use non-constant contamination rate by using the beta distribution.

In [3]:
%%writefile data/complex.yml
consignment:
  generation_method: parameter_based
  parameter_based:
    origins:
    - Netherlands
    - Mexico
    flowers:
    - Hyacinthus
    - Rosa
    ports:
    - NY JFK CBP
    - FL Miami Air CBP
    boxes:
      min: 1
      max: 3000
  items_per_box:
    default: 200
contamination:
  contamination_unit: item
  contamination_rate:
    distribution: beta
    value: 0.05
    parameters:
    - 1
    - 8
  arrangement: clustered
  clustered:
    contaminated_units_per_cluster: 200
    distribution: random
    random:
      cluster_item_width: 400
inspection:
  # Here we are using boxes as a sample unit.
  unit: boxes
  min_boxes: 0
  # Here we use a contamination tolerance level of 0.005, so we will get a
  # count of the missed consignments with contamination rates below this threshold.
  tolerance_level: 0.005
  # Here we use and configure hypergeometric sampling.
  sample_strategy: hypergeometric
  hypergeometric:
    detection_level: 0.1
    confidence_level: 0.95
  # We can keep the configuration for sample_strategy: percentage here.
  # It is not used by the simulation and we can easily switch to it by changing sample_strategy
  # without actually touching the configurations under percentage: or hypergeometric:.
  proportion:
    value: 0.02
  within_box_proportion: 1
  # We can change how sample units are selected for inspection by changing selection_strategy
  # to either random or convenience.
  selection_strategy: random

Writing data/complex.yml


Now we simulate more consignment and ask for the results to be averaged over multiple 
stochastic runs to increase robustness of the result (but we still keep both values 
low to get result quickly for this example): 

In [4]:
!python -m popsborder --num-simulations=4 --num-consignments=60 --config-file=data/complex.yml --seed=4



Simulation parameters:
----------------------------------------------------------
consignments:
	 Number consignments simulated: 60
	 Avg. number of boxes per consignment: 1,479
	 Avg. number of items per consignment: 295,863
contamination:
	 unit: item
	 type: beta
		 contamination distribution parameters: [1, 8]
	 contaminant arrangement: clustered
		 maximum contaminated items per cluster: 200 items
		 cluster distribution: random
		 cluster width: 400 items
inspection:
	 unit: boxes
	 sample strategy: hypergeometric
		 detection level: 0.1
	 selection strategy: random
	 minimum proportion of items inspected within box: 1
	 tolerance level: 0.005


Simulation results: (averaged across all simulation runs)
----------------------------------------------------------
Avg. % contaminated consignments slipped: 10.83%
Adjusted avg. % contaminated consignments slipped (excluding slipped consignments with contamination rates below tolerance level): 7.08%
Avg. num. consignments slipped: 6
A

To learn more about the configuration and the simulation itself start in the [readme file](https://github.com/ncsu-landscape-dynamics/popsborder#readme) of the simulation repository.